## fractional <---> cartesian


In [2]:
from jax_find_mic import np, jnp, frc2car, car2frc

a = jnp.asarray(np.random.rand(3))
b = jnp.asarray(np.random.rand(2, 3))
cell = jnp.asarray(np.identity(3) * 5)


%timeit  frc2car(a, cell), frc2car(b, cell)
%timeit  car2frc(a, cell), car2frc(b, cell)


21.2 µs ± 27.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
58.2 µs ± 37.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
a = jnp.asarray(np.random.rand(3))
b = jnp.asarray(np.random.rand(2, 3))
cell = jnp.asarray(np.identity(3) * 5)
a_ = a @ cell
b_ = b @ cell

print(car2frc(a_, cell) - a < 1e-15)
print(car2frc(b_, cell) - b < 1e-15)

[ True  True  True]
[[ True  True  True]
 [ True  True  True]]


## cell & pbc class

根据不同的 cell 和 pbc 情况返回不同的 jit 函数


In [4]:
from jax_find_mic import np, jnp, jax, CellWithPBC


In [5]:
from ase.geometry import find_mic


for pbc in (True, [True, False, True], False):
    cell = np.random.rand(3, 3)
    v = np.random.rand(5, 3)

    mic_1 = find_mic(v, cell, pbc)[0]
    mic_2 = CellWithPBC(cell, pbc, False).fn_mic(v)
    assert np.allclose(mic_1, mic_2)

    f = CellWithPBC(cell, pbc, True).fn_mic
    assert np.allclose(mic_1, f(v, cell))

    def ff(v, cell):
        return jnp.sum(f(v, cell))
    ff = jax.jit(ff)
    jax.jacfwd(ff, argnums=1)(v, cell)


IndexError: list index out of range